BloodCellSegmentation using CNN(100% acc)
Dataset: https://www.kaggle.com/datasets/jeetblahiri/bccd-dataset-with-mask

In [1]:
!pip install opendatasets tensorflow_io matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 53.9 MB/s eta 0:00:00


In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/jeetblahiri/bccd-dataset-with-mask')

100%|██████████| 2.15G/2.15G [02:10<00:00, 17.6MB/s]


In [3]:
import os
from PIL import Image

# Set the paths to your dataset folders
train_data_dir = '/content/bccd-dataset-with-mask/BCCD Dataset with mask/train'
test_data_dir = '/content/bccd-dataset-with-mask/BCCD Dataset with mask/test'

# Set the desired size for the images
desired_width = 256
desired_height = 256

# Function to resize and save an image
def preprocess_image(image_path, output_path):
    image = Image.open(image_path)
    resized_image = image.resize((desired_width, desired_height))
    resized_image.save(output_path)

# Preprocess the training data
for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        image_path = os.path.join(root, file)
        output_path = os.path.join(root, 'resized_' + file)
        preprocess_image(image_path, output_path)
        os.remove(image_path)  # Remove the original image

# Preprocess the testing data
for root, dirs, files in os.walk(test_data_dir):
    for file in files:
        image_path = os.path.join(root, file)
        output_path = os.path.join(root, 'resized_' + file)
        preprocess_image(image_path, output_path)
        os.remove(image_path)  # Remove the original image


In [4]:
from PIL import Image

# Set the path to one of your images
image_path = '/content/bccd-dataset-with-mask/BCCD Dataset with mask/test/mask/resized_e5fd5949-4cd0-4cb6-837a-02c26a5eb87a.png'

# Open the image using PIL
image = Image.open(image_path)

# Get the image dimensions
width, height = image.size

# Get the number of channels (e.g., 3 for RGB images)
channels = len(image.getbands())

# Print the values
print("Image Width:", width)
print("Image Height:", height)
print("Number of Channels:", channels)


Image Width: 256
Image Height: 256
Number of Channels: 3


In [5]:
import tensorflow as tf
from tensorflow.keras import layers

# Set the path to your dataset folders
train_data_dir = '/content/bccd-dataset-with-mask/BCCD Dataset with mask/train'
test_data_dir = '/content/bccd-dataset-with-mask/BCCD Dataset with mask/test'

input_shape = (256,256,3)


num_classes = 2  # mask and original

# Data preprocessing and augmentation
train_data = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load and augment the training data
train_generator = train_data.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical'
)

# Load and rescale the testing data
test_generator = test_data.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical'
)

Found 2338 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


In [6]:

# Build the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [11]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)



Epoch 1/5
74/74 [==============================] - 40s 537ms/step - loss: 9.0678e-06 - accuracy: 1.0000 - val_loss: 6.8065e-06 - val_accuracy: 1.0000
Epoch 2/5
74/74 [==============================] - 39s 532ms/step - loss: 6.6666e-06 - accuracy: 1.0000 - val_loss: 5.1213e-06 - val_accuracy: 1.0000
Epoch 3/5
74/74 [==============================] - 38s 520ms/step - loss: 5.1678e-06 - accuracy: 1.0000 - val_loss: 4.0365e-06 - val_accuracy: 1.0000
Epoch 4/5
74/74 [==============================] - 39s 523ms/step - loss: 4.1898e-06 - accuracy: 1.0000 - val_loss: 3.2901e-06 - val_accuracy: 1.0000
Epoch 5/5
74/74 [==============================] - 38s 520ms/step - loss: 3.4384e-06 - accuracy: 1.0000 - val_loss: 2.7612e-06 - val_accuracy: 1.0000


In [13]:
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

10/10 [==============================] - 1s 98ms/step - loss: 2.7612e-06 - accuracy: 1.0000
Test Loss: 2.7612322810455225e-06
Test Accuracy: 1.0
